Datasets:
- CSV 1: international_matches.csv

Columnas: ID, Tournament, Date, Home Team, Home Goals, Away Goals, Away Team, Win Conditions, Home Stadium
- CSV 2: 2022_world_cup_matches.csv
Columnas: ID, Year, Date, Stage, Home Team, Away Team, Host Team

En este ejercicio se pide combinar estos datos con información adicional de los continentes a los que pertenecen los equipos y luego asignar una probabilidad de victoria basada en los históricos.


Objetivos:
1. Cargar datos y explorarlos.
2. Limpieza y preparación de datos
3. Preparación de datos adicionales y merge
4. Cálculo de probabilidades de victoria bada en histórico
5. Análisis exploratorio.
6. Visualización.


## 1. Carga de los datos


In [2]:
import pandas as pd

# 1. Cargar el CSV de partidos internacionales
# Usamos parse_dates=['Date'] para que pandas interprete esa columna como fechas automáticamente
df_international = pd.read_csv('international_matches.csv', parse_dates=['Date'])

# 2. Cargar el CSV de partidos del mundial
# Nota: Si tu archivo local se llama '2022_world_cup_matches.csv', cambia el nombre aquí abajo.
df_world_cup = pd.read_csv('2022_world_cup_matches.csv', parse_dates=['Date'])

# 3. Revisar la estructura de 'df_international'
print("--- INFO: International Matches ---")
df_international.info()
print("\n--- HEAD: International Matches (Primeras 5 filas) ---")
print(df_international.head())

# 4. Revisar la estructura de 'df_world_cup'
print("\n" + "="*50 + "\n") # Separador visual
print("--- INFO: World Cup Matches ---")
df_world_cup.info()
print("\n--- HEAD: World Cup Matches (Primeras 5 filas) ---")
print(df_world_cup.head())

--- INFO: International Matches ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17769 entries, 0 to 17768
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ID              17769 non-null  int64         
 1   Tournament      17769 non-null  object        
 2   Date            17769 non-null  datetime64[ns]
 3   Home Team       17769 non-null  object        
 4   Home Goals      17769 non-null  int64         
 5   Away Goals      17769 non-null  int64         
 6   Away Team       17769 non-null  object        
 7   Win Conditions  201 non-null    object        
 8   Home Stadium    17769 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(3), object(4)
memory usage: 1.1+ MB

--- HEAD: International Matches (Primeras 5 filas) ---
   ID Tournament       Date Home Team  Home Goals  Away Goals Away Team  \
0   1   Friendly 1872-11-30  Scotland           0           0   England   


## 2. Limpieza y preparación de datos

In [5]:
# Conteo de valores nulos por columna
print("Valores nulos por columna (df_international):")
print(df_international.isna().sum())

# Asegurar que las columnas de goles sean numéricas
df_international["Home Goals"] = pd.to_numeric(df_international["Home Goals"], errors="coerce")
df_international["Away Goals"] = pd.to_numeric(df_international["Away Goals"], errors="coerce")

# Eliminar filas con datos faltantes críticos
df_international = df_international.dropna(subset=["Home Goals", "Away Goals", "Date"])

# Crear nuevas variables
df_international["goal_diff"] = df_international["Home Goals"] - df_international["Away Goals"]
df_international["total_goals"] = df_international["Home Goals"] + df_international["Away Goals"]

print("\nDespués de limpieza de NaN y creación de columnas:")
print(df_international.isna().sum())
display(df_international.head())

print("\n--- Revisión de columnas del CSV del Mundial ---")
print(df_world_cup.columns)

# Verificar consistencia de nombres de columnas
if "Home Team" in df_world_cup.columns and "Away Team" in df_world_cup.columns:
    print("\nColumnas correctas: 'Home Team' y 'Away Team' están presentes.")
else:
    df_world_cup = df_world_cup.rename(columns={
        "HomeTeam": "Home Team",
        "AwayTeam": "Away Team"
    })
    print("\nColumnas renombradas para mantener consistencia.")

# Comprobar valores nulos (por ejemplo en fases donde aún no hay equipos asignados)
print("\nValores nulos por columna en df_world_cup:")
print(df_world_cup.isna().sum())

# Mostrar una muestra para confirmar que todo está bien
print("\nVista previa de df_world_cup:")
display(df_world_cup.head())

Valores nulos por columna (df_international):
ID                    0
Tournament            0
Date                  0
Home Team             0
Home Goals            0
Away Goals            0
Away Team             0
Win Conditions    17568
Home Stadium          0
goal_diff             0
total_goals           0
dtype: int64

Después de limpieza de NaN y creación de columnas:
ID                    0
Tournament            0
Date                  0
Home Team             0
Home Goals            0
Away Goals            0
Away Team             0
Win Conditions    17568
Home Stadium          0
goal_diff             0
total_goals           0
dtype: int64


,ID,Tournament,Date,Home Team,Home Goals,Away Goals,Away Team,Win Conditions,Home Stadium,goal_diff,total_goals
0,1,Friendly,1872-11-30,Scotland,0,0,England,NaN,True,0,0
1,2,Friendly,1873-03-08,England,4,2,Scotland,NaN,True,2,6
2,3,Friendly,1874-03-07,Scotland,2,1,England,NaN,True,1,3
3,4,Friendly,1875-03-06,England,2,2,Scotland,NaN,True,0,4
4,5,Friendly,1876-03-04,Scotland,3,0,England,NaN,True,3,3



--- Revisión de columnas del CSV del Mundial ---
Index(['ID', 'Year', 'Date', 'Stage', 'Home Team', 'Away Team', 'Host Team'], dtype='object')

Columnas correctas: 'Home Team' y 'Away Team' están presentes.

Valores nulos por columna en df_world_cup:
ID            0
Year          0
Date          0
Stage         0
Home Team     0
Away Team     0
Host Team    16
dtype: int64

Vista previa de df_world_cup:


,ID,Year,Date,Stage,Home Team,Away Team,Host Team
0,1,2022,2022-11-20,Group stage,Qatar,Ecuador,True
1,2,2022,2022-11-21,Group stage,Senegal,Netherlands,False
2,3,2022,2022-11-21,Group stage,England,Iran,False
3,4,2022,2022-11-21,Group stage,United States,Wales,False
4,5,2022,2022-11-22,Group stage,France,Australia,False


## 3. Preparación de datos adicionales y merge

In [8]:
# Detectar filas duplicadas completas
duplicados = df[df.duplicated()]
print("Filas duplicadas detectadas:")
display(duplicados)

# Eliminar duplicados
df.drop_duplicates(inplace=True)

print("Tamaño tras eliminar duplicados:", df.shape)


Filas duplicadas detectadas:


,user_id,app_name,screen_time(min),notifications,date,comentarios
7,1002,WhatsApp,60.0,12.0,2024-05-12,NaN


Tamaño tras eliminar duplicados: (16, 6)


## 4. Cálculo de probabilidades de victoria basada en histórico

In [ ]:
print("Tipos ANTES:")
print(df.dtypes)

# Convertir columnas numéricas que puedan venir como texto
df['screen_time(min)'] = df['screen_time(min)'].astype(float)

# notifications puede tener mezcla de enteros y strings -> forzamos conversión segura
df['notifications'] = pd.to_numeric(df['notifications'], errors='coerce').fillna(0).astype(int)

# Convertir fechas a datetime, forzando formato cuando hay mezcla
df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True)

print("\nTipos DESPUÉS:")
print(df.dtypes)

display(df.head())


## 5. Análisis exploratorio

In [2]:
# Eliminamos columnas que estén completamente vacías
df.dropna(axis=1, how='all', inplace=True)

print("Columnas actuales del DataFrame:")
print(df.columns.tolist())


NameError: name 'df' is not defined

## 6. Visualización

In [ ]:
# Normalizar nombres de apps:
# - quitar espacios en blanco
# - poner la primera letra en mayúscula de cada palabra
# - eliminar espacios internos (Instagram  -> Instagram, 'YouTube Music' -> 'Youtubemusic')
df['app_name'] = (
    df['app_name']
      .astype(str)
      .str.strip()
      .str.title()
      .str.replace(' ', '', regex=False)
)

display(df[['app_name']].head(10))

# Buscar todas las filas que contienen "Youtube"
youtube_rows = df[df['app_name'].str.contains('Youtube', case=False, na=False)]
print("Filas con Youtube:")
display(youtube_rows)


In [ ]:
# Longitud del nombre de la app
df['app_name_len'] = df['app_name'].str.len()

# Apps que empiezan por 'Y'
apps_y = df[df['app_name'].str.startswith('Y', na=False)]

print("Apps que empiezan por 'Y':")
display(apps_y[['app_name', 'app_name_len']].drop_duplicates())

display(df[['app_name', 'app_name_len']].head())


In [ ]:
print("Tipos actuales:")
print(df.dtypes)

print("\nTipo real de cada celda en las primeras filas:")
display(df.applymap(type).head())

# Convertir automáticamente al tipo más adecuado (opcional)
df = df.convert_dtypes()

print("\nTipos tras convert_dtypes():")
print(df.dtypes)


In [ ]:
df.to_csv('screentime_analysis_clean.csv', index=False)
print('Archivo screentime_analysis_clean.csv guardado correctamente.')

In [3]:
# ¿Qué apps tienen más tiempo medio de pantalla?
screen_time_media = (
    df.groupby('app_name')['screen_time(min)']
      .mean()
      .sort_values(ascending=False)
)

print("Tiempo medio de pantalla (min) por app:")
display(screen_time_media)

# ¿Qué apps generan más notificaciones?
notif_media = (
    df.groupby('app_name')['notifications']
      .mean()
      .sort_values(ascending=False)
)

print("Notificaciones medias por app:")
display(notif_media)


Tiempo medio de pantalla (min) por app:


app_name
YouTube Music    130.0
YouTube          120.0
Netflix          110.0
Tik tok          100.0
YouTube Kids      95.0
  WhatsApp        80.0
WhatsApp          80.0
Telegram          60.0
Calendar Pro      50.0
Instagram         45.0
 instagram        30.0
TikTok             NaN
Name: screen_time(min), dtype: float64

Notificaciones medias por app:


app_name
  WhatsApp       15.0
Tik tok          12.0
Instagram        12.0
WhatsApp         12.0
YouTube Music     9.0
YouTube           8.0
YouTube Kids      7.0
Netflix           6.0
 instagram        6.0
Telegram          5.0
Calendar Pro      3.0
TikTok            NaN
Name: notifications, dtype: float64